In [1]:
import glob
import os
import json
import pickle
import yaml
import math
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import pyarrow.parquet as pq
#from sklearn.metrics import auc, roc_curve
#from scipy.special import softmax
import hist as hist2
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
import utils_farouk2 as utils
plt.rcParams.update({"font.size": 20})

year = "2018"
channels = ["mu", "ele"]
samplesMC = ["TTbar", "WJetsLNu", "DYJets", ]

In [2]:
samples_dir="/uscms/home/jieun201/nobackup/YOURWORKINGAREA/data_compar_may2/" 
samplesMC = [ "TTbar", "WJetsLNu", "DYJets", ]
fakeMC= utils.event_skimmer_noInference(year, channels, samples_dir, samplesMC, "all", add_qcd_score=False, add_top_score=False, add_inclusive_score=False)

luminosity 59781.96
---> Accumulating event weights.
sample DYJetsToLL_M-10to50
xsec*lum 1112542275.6
metadata {'DYJetsToLL_M-10to50': {'mc': 146, '2018': {'sumgenweight': 1444515.0, 'cutflows': {'ele': {'all': 1444515.0, 'Trigger': 1777.0, 'METFilters': 1777.0, 'GreaterTwoFatJets': 23.0, 'LeptonCut': 11.0, 'metRevertCut': 5.0, 'HEMCleaning': 4.0}, 'mu': {'all': 1444515.0, 'Trigger': 6125.0, 'METFilters': 6124.0, 'GreaterTwoFatJets': 23.0, 'LeptonCut': 11.0, 'metRevertCut': 3.0, 'HEMCleaning': 3.0}}}}}
metadata {'DYJetsToLL_M-10to50': {'mc': 580, '2018': {'sumgenweight': 5803432.0, 'cutflows': {'ele': {'all': 5803432.0, 'Trigger': 7632.0, 'METFilters': 7631.0, 'GreaterTwoFatJets': 70.0, 'LeptonCut': 35.0, 'metRevertCut': 12.0, 'HEMCleaning': 9.0}, 'mu': {'all': 5803432.0, 'Trigger': 24796.0, 'METFilters': 24791.0, 'GreaterTwoFatJets': 95.0, 'LeptonCut': 45.0, 'metRevertCut': 12.0, 'HEMCleaning': 9.0}}}}}
metadata {'DYJetsToLL_M-10to50': {'mc': 697, '2018': {'sumgenweight': 7006069.0, '

In [3]:
my_list = list(fakeMC['ele']['DYJets'])
my_list

['met_pt',
 'numberFatJet',
 'lep_pt',
 'lep_isolation',
 'lep_misolation',
 'lep_fj_dr',
 'lep_eta',
 'numberLeptons',
 'numberBJets_Medium_OutsideFatJets',
 'numberBJets_Tight_OutsideFatJets',
 'lep_pt_loose',
 'lep_isolation_loose',
 'lep_misolation_loose',
 'lep_fj_dr_loose',
 'lep_eta_loose',
 'numberLeptons_loose',
 'numberBJets_Medium_OutsideFatJets_loose',
 'numberBJets_Tight_OutsideFatJets_loose',
 'fj_minus_lep_m',
 'fj_pt',
 'rec_higgs_m',
 'weight_ele',
 'weight_mu',
 'event_weight']

In [4]:
ch = 'ele'
#good electron pt cut is at 38 GeV
bins_PT = [30,40,50,60,70,80,90,100,200,2000]
#bins_PT = [30,45,60,70,80,90,100,150,200,600]
bins_ETA = [0,0.5,1,1.5,2,2.5]

In [5]:
df_MC_tight_TT = fakeMC[ch]['TTbar']
df_MC_tight_TTbar = df_MC_tight_TT[    (df_MC_tight_TT["lep_fj_dr"] < 0.8) &  (df_MC_tight_TT["lep_fj_dr"] > 0.03) & (df_MC_tight_TT["numberLeptons"] == 1) 
& (df_MC_tight_TT["numberBJets_Medium_OutsideFatJets"] == 0)
& (   (df_MC_tight_TT['lep_pt'] < 55)  | (    (df_MC_tight_TT['lep_pt'] >= 55)  & (df_MC_tight_TT['lep_misolation'] < 0.2) )    ) 
]

df_MC_tight_WJ = fakeMC[ch]['WJetsLNu']
df_MC_tight_WJets = df_MC_tight_WJ[    (df_MC_tight_WJ["lep_fj_dr"] < 0.8) &  (df_MC_tight_WJ["lep_fj_dr"] > 0.03) & (df_MC_tight_WJ["numberLeptons"] == 1) 
& (df_MC_tight_WJ["numberBJets_Medium_OutsideFatJets"] == 0)
& (   (df_MC_tight_WJ['lep_pt'] < 55)  | (    (df_MC_tight_WJ['lep_pt'] >= 55)  & (df_MC_tight_WJ['lep_misolation'] < 0.2) )    ) 
]

df_MC_tight_DY = fakeMC[ch]['DYJets']
df_MC_tight_DYJets = df_MC_tight_DY[    (df_MC_tight_DY["lep_fj_dr"] < 0.8) &  (df_MC_tight_DY["lep_fj_dr"] > 0.03) & (df_MC_tight_DY["numberLeptons"] == 1) 
& (df_MC_tight_DY["numberBJets_Medium_OutsideFatJets"] == 0)
& (   (df_MC_tight_DY['lep_pt'] < 55)  | (    (df_MC_tight_DY['lep_pt'] >= 55)  & (df_MC_tight_DY['lep_misolation'] < 0.2) )    ) 
]

In [6]:
df_MC_tight = pd.concat([df_MC_tight_TTbar,df_MC_tight_WJets,df_MC_tight_DYJets])

In [7]:
df_MC_tight['event_weight'].sum()

33867.72816687235

In [8]:
grouped_MC_tight = df_MC_tight.groupby(
        [
        pd.cut(df_MC_tight["lep_pt"], bins_PT),
             pd.cut(np.abs(df_MC_tight["lep_eta"]), bins_ETA),
    ]
#    observed=False , # Added this because the default will change in the future
)
grouped_MC_tight['event_weight'].sum()  


lep_pt       lep_eta   
(30, 40]     (0.0, 0.5]      65.067088
             (0.5, 1.0]      57.160273
             (1.0, 1.5]      45.945571
             (1.5, 2.0]      41.894940
             (2.0, 2.5]      24.015308
(40, 50]     (0.0, 0.5]     373.278988
             (0.5, 1.0]     384.469873
             (1.0, 1.5]     277.879505
             (1.5, 2.0]     242.275832
             (2.0, 2.5]     186.865011
(50, 60]     (0.0, 0.5]     467.253635
             (0.5, 1.0]     444.896980
             (1.0, 1.5]     376.731109
             (1.5, 2.0]     298.429404
             (2.0, 2.5]     181.768840
(60, 70]     (0.0, 0.5]     514.812189
             (0.5, 1.0]     481.355492
             (1.0, 1.5]     351.764980
             (1.5, 2.0]     346.520410
             (2.0, 2.5]     192.788795
(70, 80]     (0.0, 0.5]     510.600352
             (0.5, 1.0]     468.287037
             (1.0, 1.5]     372.547246
             (1.5, 2.0]     307.875367
             (2.0, 2.5]     197.800773
(

In [9]:
df_MC_loose_TT = fakeMC[ch]['TTbar']
df_MC_loose_TTbar = df_MC_loose_TT[    (df_MC_loose_TT["lep_fj_dr_loose"] < 0.8) &  (df_MC_loose_TT["lep_fj_dr_loose"] > 0.03) & (df_MC_loose_TT["numberLeptons_loose"] == 1) 
& (df_MC_loose_TT["numberBJets_Medium_OutsideFatJets_loose"] == 0)]

df_MC_loose_WJ = fakeMC[ch]['WJetsLNu']
df_MC_loose_WJets = df_MC_loose_WJ[    (df_MC_loose_WJ["lep_fj_dr_loose"] < 0.8) &  (df_MC_loose_WJ["lep_fj_dr_loose"] > 0.03) & (df_MC_loose_WJ["numberLeptons_loose"] == 1) 
& (df_MC_loose_WJ["numberBJets_Medium_OutsideFatJets_loose"] == 0)]

df_MC_loose_DY = fakeMC[ch]['DYJets']
df_MC_loose_DYJets = df_MC_loose_DY[    (df_MC_loose_DY["lep_fj_dr_loose"] < 0.8) &  (df_MC_loose_DY["lep_fj_dr_loose"] > 0.03) & (df_MC_loose_DY["numberLeptons_loose"] == 1)   
& (df_MC_loose_DY["numberBJets_Medium_OutsideFatJets_loose"] == 0)]

df_MC_loose = pd.concat([df_MC_loose_TTbar,df_MC_loose_WJets,df_MC_loose_DYJets])

In [10]:
grouped_MC_loose = df_MC_loose.groupby(
        [
        pd.cut(df_MC_loose["lep_pt_loose"], bins_PT),
            pd.cut(np.abs(df_MC_loose["lep_eta_loose"]), bins_ETA),
    ]
#    observed=False , # Added this because the default will change in the future
)
grouped_MC_loose['event_weight'].sum()  


lep_pt_loose  lep_eta_loose
(30, 40]      (0.0, 0.5]         77.730543
              (0.5, 1.0]         69.025584
              (1.0, 1.5]         55.435514
              (1.5, 2.0]         50.265778
              (2.0, 2.5]         29.816035
(40, 50]      (0.0, 0.5]        428.562983
              (0.5, 1.0]        444.336757
              (1.0, 1.5]        323.654162
              (1.5, 2.0]        285.461444
              (2.0, 2.5]        224.079126
(50, 60]      (0.0, 0.5]        525.756074
              (0.5, 1.0]        498.720533
              (1.0, 1.5]        417.998230
              (1.5, 2.0]        344.715034
              (2.0, 2.5]        223.215428
(60, 70]      (0.0, 0.5]        563.800211
              (0.5, 1.0]        550.162716
              (1.0, 1.5]        393.924122
              (1.5, 2.0]        389.104603
              (2.0, 2.5]        232.188240
(70, 80]      (0.0, 0.5]        551.571564
              (0.5, 1.0]        518.351220
              (1.0, 1.5]  

In [11]:
grouped_MC_tight['event_weight'].sum()/grouped_MC_loose['event_weight'].sum()      #sanity check that tight is less than loose

lep_pt       lep_eta   
(30, 40]     (0.0, 0.5]    0.837085
             (0.5, 1.0]    0.828103
             (1.0, 1.5]    0.828811
             (1.5, 2.0]    0.833468
             (2.0, 2.5]    0.805449
(40, 50]     (0.0, 0.5]    0.871001
             (0.5, 1.0]    0.865267
             (1.0, 1.5]    0.858569
             (1.5, 2.0]    0.848716
             (2.0, 2.5]    0.833924
(50, 60]     (0.0, 0.5]    0.888727
             (0.5, 1.0]    0.892077
             (1.0, 1.5]    0.901274
             (1.5, 2.0]    0.865728
             (2.0, 2.5]    0.814320
(60, 70]     (0.0, 0.5]    0.913111
             (0.5, 1.0]    0.874933
             (1.0, 1.5]    0.892976
             (1.5, 2.0]    0.890558
             (2.0, 2.5]    0.830312
(70, 80]     (0.0, 0.5]    0.925719
             (0.5, 1.0]    0.903416
             (1.0, 1.5]    0.902206
             (1.5, 2.0]    0.883839
             (2.0, 2.5]    0.840852
(80, 90]     (0.0, 0.5]    0.931961
             (0.5, 1.0]    0.910568
    

# this is for DATA

In [12]:
samples_dir="/uscms/home/jieun201/nobackup/YOURWORKINGAREA/data_compar_may2/" 
samplesData = [ "Data", ]
fakesRate_Data= utils.event_skimmer_noInference(year, channels, samples_dir, samplesData, "all", add_qcd_score=False, add_top_score=False, add_inclusive_score=False)

luminosity 59781.96
ATTENTION!: DYJetsToLL_M-10to50 will be skipped
ATTENTION!: DYJetsToLL_M-50_HT-100to200 will be skipped
ATTENTION!: DYJetsToLL_M-50_HT-1200to2500 will be skipped
ATTENTION!: DYJetsToLL_M-50_HT-200to400 will be skipped
ATTENTION!: DYJetsToLL_M-50_HT-2500toInf will be skipped
ATTENTION!: DYJetsToLL_M-50_HT-400to600 will be skipped
ATTENTION!: DYJetsToLL_M-50_HT-600to800 will be skipped
ATTENTION!: DYJetsToLL_M-50_HT-70to100 will be skipped
ATTENTION!: DYJetsToLL_M-50_HT-800to1200 will be skipped
ATTENTION!: TTTo2L2Nu will be skipped
ATTENTION!: TTToHadronic will be skipped
ATTENTION!: TTToSemiLeptonic will be skipped
ATTENTION!: WJetsToLNu_HT-100To200 will be skipped
ATTENTION!: WJetsToLNu_HT-1200To2500 will be skipped
ATTENTION!: WJetsToLNu_HT-200To400 will be skipped
ATTENTION!: WJetsToLNu_HT-2500ToInf will be skipped
ATTENTION!: WJetsToLNu_HT-400To600 will be skipped
ATTENTION!: WJetsToLNu_HT-600To800 will be skipped
ATTENTION!: WJetsToLNu_HT-800To1200 will be skip

In [13]:
df_Data_Tight = fakesRate_Data[ch]["Data"]
df_Data_Tight = df_Data_Tight[ (df_Data_Tight["lep_fj_dr"] < 0.8) &  (df_Data_Tight["lep_fj_dr"] > 0.03) & (df_Data_Tight["numberLeptons"] == 1)
& (df_Data_Tight["numberBJets_Medium_OutsideFatJets"] == 0)
& (   (df_Data_Tight['lep_pt'] < 55)  | (    (df_Data_Tight['lep_pt'] >= 55)  & (df_Data_Tight['lep_misolation'] < 0.2) )    )
]

grouped_Data_Tight = df_Data_Tight.groupby(
        [
        pd.cut(df_Data_Tight["lep_pt"], bins_PT),
            pd.cut(np.abs(df_Data_Tight["lep_eta"]), bins_ETA),
    ]
#    observed=False , # Added this because the default will change in the future
)
grouped_Data_Tight['event_weight'].sum()  


lep_pt       lep_eta   
(30, 40]     (0.0, 0.5]      82.0
             (0.5, 1.0]      93.0
             (1.0, 1.5]      64.0
             (1.5, 2.0]      75.0
             (2.0, 2.5]      83.0
(40, 50]     (0.0, 0.5]     406.0
             (0.5, 1.0]     382.0
             (1.0, 1.5]     352.0
             (1.5, 2.0]     520.0
             (2.0, 2.5]     364.0
(50, 60]     (0.0, 0.5]     463.0
             (0.5, 1.0]     496.0
             (1.0, 1.5]     413.0
             (1.5, 2.0]     481.0
             (2.0, 2.5]     408.0
(60, 70]     (0.0, 0.5]     504.0
             (0.5, 1.0]     494.0
             (1.0, 1.5]     420.0
             (1.5, 2.0]     542.0
             (2.0, 2.5]     394.0
(70, 80]     (0.0, 0.5]     487.0
             (0.5, 1.0]     483.0
             (1.0, 1.5]     437.0
             (1.5, 2.0]     508.0
             (2.0, 2.5]     369.0
(80, 90]     (0.0, 0.5]     552.0
             (0.5, 1.0]     471.0
             (1.0, 1.5]     413.0
             (1.5, 2.0] 

In [14]:
df_Data_loose = fakesRate_Data[ch]["Data"]
df_Data_loose = df_Data_loose[ (df_Data_loose["lep_fj_dr_loose"] < 0.8) &  (df_Data_loose["lep_fj_dr_loose"] > 0.03) & (df_Data_loose["numberLeptons_loose"] == 1) &
(df_Data_loose["numberBJets_Medium_OutsideFatJets_loose"] == 0)
#& (   (df_Data_loose['lep_pt_loose'] < 55)  | (    (df_Data_loose['lep_pt_loose'] >= 55)  & (df_Data_loose['lep_misolation_loose'] < 0.2) )    )
]
#df_Data_loose['lep_pt'].hist()

In [15]:
grouped_Data_loose = df_Data_loose.groupby(
        [
        pd.cut(df_Data_loose["lep_pt_loose"], bins_PT),
              pd.cut(np.abs(df_Data_loose["lep_eta_loose"]), bins_ETA),
    ]
#    observed=False , # Added this because the default will change in the future
)
grouped_Data_loose['event_weight'].sum()  


lep_pt_loose  lep_eta_loose
(30, 40]      (0.0, 0.5]         419.0
              (0.5, 1.0]         421.0
              (1.0, 1.5]         365.0
              (1.5, 2.0]         346.0
              (2.0, 2.5]         321.0
(40, 50]      (0.0, 0.5]        1682.0
              (0.5, 1.0]        1694.0
              (1.0, 1.5]        1657.0
              (1.5, 2.0]        1816.0
              (2.0, 2.5]        1426.0
(50, 60]      (0.0, 0.5]        1375.0
              (0.5, 1.0]        1423.0
              (1.0, 1.5]        1356.0
              (1.5, 2.0]        1598.0
              (2.0, 2.5]        1345.0
(60, 70]      (0.0, 0.5]        1328.0
              (0.5, 1.0]        1322.0
              (1.0, 1.5]        1224.0
              (1.5, 2.0]        1539.0
              (2.0, 2.5]        1239.0
(70, 80]      (0.0, 0.5]        1216.0
              (0.5, 1.0]        1254.0
              (1.0, 1.5]        1209.0
              (1.5, 2.0]        1392.0
              (2.0, 2.5]        1052

In [20]:
FakeRates_EleWithoutSubtraction = (grouped_Data_Tight['event_weight'].sum())/(grouped_Data_loose['event_weight'].sum())
FakeRates_EleWithoutSubtraction

lep_pt       lep_eta   
(30, 40]     (0.0, 0.5]    0.195704
             (0.5, 1.0]    0.220903
             (1.0, 1.5]    0.175342
             (1.5, 2.0]    0.216763
             (2.0, 2.5]    0.258567
(40, 50]     (0.0, 0.5]    0.241379
             (0.5, 1.0]    0.225502
             (1.0, 1.5]    0.212432
             (1.5, 2.0]    0.286344
             (2.0, 2.5]    0.255259
(50, 60]     (0.0, 0.5]    0.336727
             (0.5, 1.0]    0.348559
             (1.0, 1.5]    0.304572
             (1.5, 2.0]    0.301001
             (2.0, 2.5]    0.303346
(60, 70]     (0.0, 0.5]    0.379518
             (0.5, 1.0]    0.373676
             (1.0, 1.5]    0.343137
             (1.5, 2.0]    0.352177
             (2.0, 2.5]    0.317998
(70, 80]     (0.0, 0.5]    0.400493
             (0.5, 1.0]    0.385167
             (1.0, 1.5]    0.361456
             (1.5, 2.0]    0.364943
             (2.0, 2.5]    0.350760
(80, 90]     (0.0, 0.5]    0.473008
             (0.5, 1.0]    0.420912
    

In [21]:
FakeRates_Ele = (grouped_Data_Tight['event_weight'].sum()-(grouped_MC_tight['event_weight'].sum()))/(grouped_Data_loose['event_weight'].sum()  -grouped_MC_loose['event_weight'].sum() )

In [22]:
FakeRates_Ele

lep_pt       lep_eta   
(30, 40]     (0.0, 0.5]    0.049617
             (0.5, 1.0]    0.101825
             (1.0, 1.5]    0.058322
             (1.5, 2.0]    0.111942
             (2.0, 2.5]    0.202568
(40, 50]     (0.0, 0.5]    0.026105
             (0.5, 1.0]   -0.001976
             (1.0, 1.5]    0.055590
             (1.5, 2.0]    0.181455
             (2.0, 2.5]    0.147377
(50, 60]     (0.0, 0.5]   -0.005009
             (0.5, 1.0]    0.055290
             (1.0, 1.5]    0.038666
             (1.5, 2.0]    0.145674
             (2.0, 2.5]    0.201671
(60, 70]     (0.0, 0.5]   -0.014148
             (0.5, 1.0]    0.016382
             (1.0, 1.5]    0.082203
             (1.5, 2.0]    0.169998
             (2.0, 2.5]    0.199850
(70, 80]     (0.0, 0.5]   -0.035520
             (0.5, 1.0]    0.020000
             (1.0, 1.5]    0.080964
             (1.5, 2.0]    0.191752
             (2.0, 2.5]    0.209607
(80, 90]     (0.0, 0.5]    0.125923
             (0.5, 1.0]    0.005477
    

In [18]:
FakeRates_Ele.to_csv('fakeRatesEle.csv')